<a href="https://colab.research.google.com/github/la2015-hw/Group_10/blob/main/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mohamedihsanfazal","key":"9bfefc096570019378ac178fc7dbeb89"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d vaishnavichintha/sdss-tabular-dataset

Dataset URL: https://www.kaggle.com/datasets/vaishnavichintha/sdss-tabular-dataset
License(s): unknown
  0% 0.00/10.7M [00:00<?, ?B/s]
100% 10.7M/10.7M [00:00<00:00, 1.06GB/s]


In [4]:
!unzip sdss-tabular-dataset.zip

Archive:  sdss-tabular-dataset.zip
  inflating: sdss_gz2_matched_cleaned_balanced.csv  


In [42]:
import pandas as pd
import numpy as np

df = pd.read_csv("sdss_gz2_matched_cleaned_balanced.csv")


In [45]:
from sklearn.model_selection import train_test_split

# Selecting Target column
y = df["broad_class"]

# Selecting Features, unnecessary columns have been dropped
x = df.drop(columns=["specobjid", "dr8objid", "dr7objid",
    "ra", "dec", "rastring", "decstring",
    "sample", "gz2class", "broad_class"])

# Computing correlation matrix
corr_matrix = x.corr()

# Select upper triangle to avoid duplicate checks
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Keep only features that have correlation > 0.5 or < -0.5 with any other feature
to_keep = [column for column in upper.columns if any(upper[column] > 0.5) or any(upper[column] < -0.5)]

# Drop features that are not in to_keep
x_reduced = x[to_keep]

print(f"Remaining features: {x_reduced.shape[1]}")

# Splitting for training and testing (70-30)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=54, stratify=y)

Remaining features: 182


In [46]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initializing Naive Bayes Model
nb = GaussianNB(var_smoothing=1e-6)

# Training the Model
nb.fit(x_train, y_train)

# Predicting on test set
y_trainpred = nb.predict(x_train)
y_testpred = nb.predict(x_test)

# Evaluating Perfomance
accuracy_training = accuracy_score(y_train, y_trainpred)
print(f"Train Accuracy: {accuracy_training:.4f}")

accuracy_test = accuracy_score(y_test, y_testpred)
print(f"Test Accuracy: {accuracy_test:.4f}")

print("\nClassification Report:\n", classification_report(y_test, y_testpred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_testpred))

Train Accuracy: 0.8276
Test Accuracy: 0.8352

Classification Report:
                  precision    recall  f1-score   support

     Elliptical       0.73      0.92      0.81      3570
      Irregular       0.88      0.97      0.92      3570
  Spiral-barred       0.90      0.82      0.86      3570
Spiral-unbarred       0.87      0.63      0.73      3570

       accuracy                           0.84     14280
      macro avg       0.85      0.84      0.83     14280
   weighted avg       0.85      0.84      0.83     14280


Confusion Matrix:
 [[3279  154   17  120]
 [  69 3456    7   38]
 [ 327  135 2945  163]
 [ 846  192  286 2246]]
